In [1]:
import pandas as pd

In [2]:
# First, let's reload the original datasets since the previous dataframes have been altered through oversampling
orig_data = pd.read_csv('/Users/filippoorlandi/Desktop/ARTICLE/Instructions_Article/BRASILE /fake_original_BRAZIL.csv')
false_data = pd.read_csv('/Users/filippoorlandi/Desktop/ARTICLE/Instructions_Article/BRASILE /original_log_br.csv')

# Rename the columns for clarity
orig_data.rename(columns={'0': 'Log_Return'}, inplace=True)
false_data.rename(columns={'0': 'Log_Return'}, inplace=True)
orig_data.drop(orig_data.tail(1).index, inplace=True)

In [3]:
df = pd.DataFrame(orig_data)
df ['Fake_Log_Return'] = false_data
df

,Log_Return,Fake_Log_Return
0,0.020506,-0.065855
1,-0.018665,0.024553
2,0.004328,-0.008531
3,-0.017426,0.012463
4,0.012386,0.042790
...,...,...
2294,0.015233,0.021954
2295,-0.005720,-0.027134
2296,0.013064,0.042732
2297,-0.017426,-0.001151


In [4]:
df ['Fake_Log_Return']

0      -0.065855
1       0.024553
2      -0.008531
3       0.012463
4       0.042790
          ...   
2294    0.021954
2295   -0.027134
2296    0.042732
2297   -0.001151
2298   -0.010342
Name: Fake_Log_Return, Length: 2299, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
def prepare_lstm_data_with_extra_feature(df, n_steps, feature_name):
    X, y = [], []
    for i in range(len(df) - n_steps):
        # Extract the sequence of log returns
        log_returns_sequence = df['Log_Return'][i:i + n_steps].values.reshape(-1, 1)
        # Extract the corresponding sequence of the extra feature
        extra_feature_sequence = df[feature_name][i:i + n_steps].values.reshape(-1, 1)
        # Combine the sequences
        sequence_combined = np.hstack((log_returns_sequence, extra_feature_sequence))
        X.append(sequence_combined)
        y.append(df['Log_Return'][i + n_steps])
    return np.array(X), np.array(y)

# Example usage:
# Let's say the extra feature is in a column named 'Extra_Feature'
n_steps = 10
feature_name = 'Fake_Log_Return'

# Normalize both 'Log_Return' and 'Extra_Feature'
scaler = MinMaxScaler(feature_range=(0, 1))
prova_log_returns = df['Log_Return'].values.reshape(-1, 1)
extra_feature = df[feature_name].values.reshape(-1, 1)

prova_scaled = scaler.fit_transform(prova_log_returns)
extra_feature_scaled = scaler.fit_transform(extra_feature)

# Combine the scaled features into a single DataFrame
combined_df = pd.DataFrame({
    'Log_Return': prova_scaled.flatten(),
    'Fake_Log_Return': extra_feature_scaled.flatten()
})

# Prepare data for LSTM
X_prova, y_prova = prepare_lstm_data_with_extra_feature(combined_df, n_steps, feature_name)

# Split the data into training and testing sets
X_train_prova, X_test_prova, y_train_prova, y_test_prova = train_test_split(X_prova, y_prova, test_size=0.01, random_state=42)

X_train_prova.shape, X_test_prova.shape, y_train_prova.shape, y_test_prova.shape


((2266, 10, 2), (23, 10, 2), (2266,), (23,))

In [6]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [7]:
input_shape = (10,2)
#input_shape= (X_train_prova[1],X_train_prova[2])
model = create_lstm_model(input_shape)
history = model.fit(X_train_prova, y_train_prova, epochs=100, batch_size=32, validation_data=(X_test_prova, y_test_prova))


Epoch 1/100


/Users/filippoorlandi/Desktop/ARTICLE/Instructions_Article/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0699 - val_loss: 0.0015
Epoch 2/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0071 - val_loss: 0.0017
Epoch 3/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0065 - val_loss: 0.0013
Epoch 4/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0065 - val_loss: 0.0015
Epoch 5/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0066 - val_loss: 0.0013
Epoch 6/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0064 - val_loss: 0.0013
Epoch 7/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0066 - val_loss: 0.0014
Epoch 8/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0061 - val_loss: 0.0012
Epoch 9/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0068 - val_loss: 0.0012
Epoch 10/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0058 - val_loss: 0.0012
Epoch 11/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0067 - val_loss: 0.0012
Epoch 12/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0060 - val_lo

In [8]:
predicted_prova = model.predict(X_test_prova)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


In [9]:
from sklearn.metrics import mean_squared_error
predicted_prova = predicted_prova.reshape(-1, 1)
y_test_prova = y_test_prova.reshape(-1, 1)

# Inverse transform the predictions and actual values to their original scale
predicted_prova_inversed = scaler.inverse_transform(predicted_prova)
y_test_prova_inversed = scaler.inverse_transform(y_test_prova)

# Calculate MSE
mse_prova = mean_squared_error(y_test_prova_inversed, predicted_prova_inversed)
print("Mean Squared Error for Prova dataset:", mse_prova)


Mean Squared Error for Prova dataset: 8.384752812069052e-05


In [10]:
from math import sqrt
rmse = sqrt (mse_prova)
rmse

0.009156829588929266

In [11]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test_prova_inversed, predicted_prova_inversed)

0.006495735242507234

In [12]:
# Generate predictions for the training set
predicted_train_prova = model.predict(X_train_prova)


71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [13]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Inverse transform the predictions and actual values to their original scale
predicted_train_prova_inv = scaler.inverse_transform(predicted_train_prova)
y_train_prova_inv = scaler.inverse_transform(y_train_prova.reshape(-1, 1))

# Calculate mean and standard deviation of the actual values
mu = np.mean(y_train_prova_inv)
sigma = np.std(y_train_prova_inv)

# Identify indices of extreme values
extreme_indices = np.where((y_train_prova_inv < (mu - 2 * sigma)) | (y_train_prova_inv > (mu + 2 * sigma)))[0]

# Filter actual and predicted values for extreme values
actual_extreme_values = y_train_prova_inv[extreme_indices]
predicted_extreme_values = predicted_train_prova_inv[extreme_indices]

# Calculate RMSE for extreme values
rmse_extreme = np.sqrt(mean_squared_error(actual_extreme_values, predicted_extreme_values))

print(f'RMSE for Extreme Values: {rmse_extreme}')


RMSE for Extreme Values: 0.058535355791874286


In [14]:
mean_absolute_error(actual_extreme_values, predicted_extreme_values)

0.05522512674588524

In [15]:
# Calculate mean and standard deviation of the actual values
mu = np.mean(y_test_prova_inversed)
sigma = np.std(y_test_prova_inversed)
# Identify indices of extreme values
extreme_indices = np.where((y_test_prova_inversed < (mu - 2 * sigma)) | (y_test_prova_inversed > (mu + 2 * sigma)))[0]

# Filter actual and predicted values for extreme values
actual_extreme_values = y_test_prova_inversed[extreme_indices]
predicted_extreme_values = predicted_prova_inversed[extreme_indices]

# Calculate RMSE for extreme values
rmse_extreme = np.sqrt(mean_squared_error(actual_extreme_values, predicted_extreme_values))

print(f'RMSE for Extreme Values: {rmse_extreme}')

RMSE for Extreme Values: 0.023469199428921304


In [16]:
mean_absolute_error(actual_extreme_values, predicted_extreme_values)

0.02342549770831647